In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sequences import *

In [2]:
time_origin = pd.to_datetime("1997-01-01T00:00:00").to_numpy()
catalog = pd.read_csv("japanese-cat.csv", sep=" ", parse_dates=["Origin_Time(UT)"])
times = catalog["Origin_Time(UT)"].to_numpy()
times_days = (times - time_origin) / np.timedelta64(1, 'D')
times = times_days.astype(np.float32)
magnitudes = catalog["JMA_Magnitude(Mj)"].to_numpy()
latitudes = catalog["Latitude(deg)"].to_numpy()
longitudes = catalog["Longitude(deg)"].to_numpy()
features = np.vstack([magnitudes, latitudes, longitudes]).T

In [20]:
subsequence_arrival_times, subsequence_features, t_start, t_end = extract_subsequences(
    times,
    features,
    max_num_sequences = 100,
    duration_scale = 30
)

subsequence_list = [ np.vstack((subsequence_arrival_times[i], subsequence_features[i].T)).T for i in range(len(subsequence_features))]



packed_features, masks = pack_sequences(
    subsequence_list, batch_first=True, valid_mask_is_true=False) 

In [22]:
import torch
from torch.nn import TransformerEncoderLayer, TransformerDecoderLayer

In [23]:
class SequenceEncoder(torch.nn.Module):
    def __init__(self, feature_dim: int, emb_dim: int, nhead: int, dim_feedforward: int, num_layers: int):
        super(SequenceEncoder, self).__init__()
        self.emb_layer = torch.nn.Linear(feature_dim, emb_dim)
        encoder_layer = TransformerEncoderLayer(d_model=emb_dim, nhead=nhead, dim_feedforward=dim_feedforward, batch_first=True)
        self.transformer_encoder = torch.nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

    def forward(self, src: torch.Tensor, src_key_padding_mask: torch.Tensor = None):
        src = self.emb_layer(src)
        casual_mask = torch.nn.Transformer.generate_square_subsequent_mask(src.shape[1], device=src.device)
        output = self.transformer_encoder(src, is_causal=True, mask=casual_mask, src_key_padding_mask=src_key_padding_mask)
        return output

In [24]:
encoder = SequenceEncoder(feature_dim=4, emb_dim=16, nhead=2, dim_feedforward=64, num_layers=1)

In [25]:
output = encoder(torch.tensor(packed_features, dtype=torch.float32), 
                 torch.tensor(masks))

/Users/schimmenti/anaconda3/envs/organoids/lib/python3.13/site-packages/torch/nn/functional.py:5962: UserWarning: Support for mismatched src_key_padding_mask and mask is deprecated. Use same type for both instead.
  warnings.warn(
